An example of how the `output_distribution_function` argument can be used to specify where a returned token should go when we do not want to or can not use types to indicate the destination place.

In [ ]:
from typing import Optional
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from pydantic import BaseModel

from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

In [ ]:
class Rubber(BaseModel):
    quantity: int


class Paint(BaseModel):
    color: str


class Ball(BaseModel):
    color: str
    size: int

In [ ]:
def make_a_ball(rubber: Rubber, paint: Paint) -> Ball:
    return Ball(color=paint.color, size=rubber.quantity * 10)

def distribute_by_colour(ball: Ball) -> dict[str, Ball]:
    if ball.color == "red":
        return {"Red Balls": ball}
    elif ball.color == "blue":
        return {"Blue Balls": ball}
    else:
        return {"Other Balls": ball}

In [ ]:
paint_inputs = [
    Paint(color="red"),
    Paint(color="blue"),
    Paint(color="green"),
    Paint(color="yellow"),
    Paint(color="red"),
    Paint(color="blue"),
    Paint(color="green"),
    Paint(color="yellow"),
    Paint(color="red"),
]
rubber_inputs = [Rubber(quantity=i) for i in range(1, 11)]

In [ ]:
executable_graph_nodes_and_edges = [
    ListPlaceNode(name="Input Rubber", type=Rubber, tokens=rubber_inputs),
    ListPlaceNode(name="Input Paint", type=Paint, tokens=paint_inputs),

    ArgumentEdgeToTransition("Input Rubber", "Make a Ball", "rubber"),
    ArgumentEdgeToTransition("Input Paint", "Make a Ball", "paint"),
    FunctionTransitionNode(
        name="Make a Ball",
        function=make_a_ball,
        output_distribution_function=distribute_by_colour
    ),
    ReturnedEdgeFromTransition("Make a Ball", "Red Balls"),
    ReturnedEdgeFromTransition("Make a Ball", "Blue Balls"),
    ReturnedEdgeFromTransition("Make a Ball", "Other Balls"),

    ListPlaceNode(name="Red Balls", type=Ball),
    ListPlaceNode(name="Blue Balls", type=Ball),
    ListPlaceNode(name="Other Balls", type=Ball),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
display(SimpleGraphvizVisualization.graph(executable_pydigraph))

In [ ]:
async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    if not transitions_fired:
        print("No more transitions to fire. Execution complete.")
        break
    time.sleep(1.0)
    plt.close()